In [1]:
import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(usecwd=True), override=True)

True

# CSV File Analysis

In [2]:
#pip install langchain_experimental

# 1) Prepare a demo CSV
import pandas as pd
df = pd.DataFrame({
    "name": ["Alice","Bob","Charlie","Diana","Ethan","Fatima"],
    "department": ["AI","Data","Cloud","AI","Data","Cloud"],
    "salary": [120000, 95000, 110000, 130000, 105000, 125000],
    "years_exp": [6, 4, 8, 9, 5, 7]
})
df.to_csv("employees.csv", index=False)

# 2) Build a CSV agent
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_csv_agent  # ✅ LC 0.3

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
csv_agent = create_csv_agent(
    llm,
    "employees.csv",
    verbose=True,
    allow_dangerous_code=True,       # <-- REQUIRED
    include_df_in_prompt=False,      # keeps prompts small
    number_of_head_rows=5            # optional: small preview
)

# 3) Ask analysis questions
#print(csv_agent.invoke({"input": "Show basic stats (count, mean) for salary and years_exp."})["output"])
print(csv_agent.invoke({"input": "Which department has the highest average salary? Return dept and value."})["output"])
#print(csv_agent.invoke({"input": "Give top-3 employees by salary as a table with name, department, salary."})["output"])
#print(csv_agent.invoke({"input": "Plot-ready summary: average salary by department as CSV (department,avg_salary)."})["output"])




> Entering new AgentExecutor chain...
Thought: To find the department with the highest average salary, I need to group the dataframe by the department, calculate the average salary for each department, and then identify the department with the maximum average salary. I will check the structure of the dataframe first to ensure I have the correct column names for department and salary. 
Action: python_repl_ast
Action Input: df.head()      name department  salary  years_exp
0    Alice         AI  120000          6
1      Bob       Data   95000          4
2  Charlie      Cloud  110000          8
3    Diana         AI  130000          9
4    Ethan       Data  105000          5The dataframe has the columns `name`, `department`, `salary`, and `years_exp`. I will group the dataframe by the `department` column, calculate the average salary for each department, and then find the department with the highest average salary.

Action: python_repl_ast  
Action Input: df.groupby('department')['salar

In [3]:
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.openai_functions_agent.base import create_openai_functions_agent
from langchain.agents import AgentExecutor

# 1) Prepare a demo CSV
import pandas as pd
df = pd.DataFrame({
    "name": ["Alice","Bob","Charlie","Diana","Ethan","Fatima"],
    "department": ["AI","Data","Cloud","AI","Data","Cloud"],
    "salary": [120000, 95000, 110000, 130000, 105000, 125000],
    "years_exp": [6, 4, 8, 9, 5, 7]
})
df.to_csv("employees.csv", index=False)

df = pd.read_csv("employees.csv")

@tool
def list_columns() -> list[str]:
    """Return CSV column names."""
    return list(df.columns)

@tool
def head(n: int = 5) -> str:
    """Return the first n rows as a table string."""
    return df.head(n).to_string(index=False)

@tool
def describe_numeric() -> str:
    """Return pandas describe() for numeric columns."""
    return df.describe().to_string()

@tool
def groupby_mean(column: str, by: str) -> str:
    """Return mean of `column` grouped by `by` as CSV text."""
    if column not in df.columns or by not in df.columns:
        return f"Column not found. Available: {list(df.columns)}"
    out = df.groupby(by)[column].mean().reset_index().sort_values(column, ascending=False)
    return out.to_csv(index=False)

tools = [list_columns, head, describe_numeric, groupby_mean]

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You analyze a local CSV using only the provided tools. "
     "Do not execute arbitrary code. If a tool is insufficient, say so."),
    MessagesPlaceholder("agent_scratchpad"),
    ("human", "{input}")
])

agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

#print(executor.invoke({"input": "What columns exist? Then show first 3 rows."})["output"])
print(executor.invoke({"input": "Mean salary by department (highest first) as CSV."})["output"])


ModuleNotFoundError: No module named 'langchain_core.memory'